<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/2_rgb_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prepare utils file

!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-14 16:28:52--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-14 16:28:52 (122 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
# Get shuffled train and validation dataset with one-hot labels

train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
# Get class weights which have calculated from the previous notebook and stored in github

classes, class_weight = get_class_weight()

In [8]:
for color, weight in zip(classes, class_weight.values()):
  print(color, "    \t=>", weight)

beige     	=> 17.94
black     	=> 13.8
blue     	=> 15.47
brown     	=> 17.92
cream     	=> 56.81
crimson     	=> 76.43
gold     	=> 56.81
green     	=> 21.2
grey     	=> 12.41
navy-blue     	=> 58.77
orange     	=> 22.37
red     	=> 16.39
silver     	=> 20.17
titanium     	=> 56.81
white     	=> 2.97
yellow     	=> 20.68


In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
rgb_creator = ModelCreator(HUB_URL, 'RGB-model')
rgb_model = rgb_creator.make_model(img_size=IMG_SIZE)

Model: "RGB-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
Total params: 23,533,136
Trainable params: 32,784
Non-trainable params: 23,500,352
_________________________________________________________________


In [11]:
# Callbacks are EarlyStopping and ModelCheckpoint

callbacks = rgb_creator.get_callbacks()

In [12]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        class_weight=class_weight)

Epoch 1/10
533/533 [==============================] - 1922s 3s/step - loss: 115.5075 - auc: 0.5076 - accuracy: 0.6004
Epoch 2/10
533/533 [==============================] - 153s 285ms/step - loss: 72.1814 - auc: 0.6392 - accuracy: 0.7226
Epoch 3/10
533/533 [==============================] - 154s 286ms/step - loss: 59.3923 - auc: 0.6988 - accuracy: 0.7675
Epoch 4/10
533/533 [==============================] - 153s 286ms/step - loss: 54.1022 - auc: 0.7338 - accuracy: 0.7939
Epoch 5/10
533/533 [==============================] - 153s 285ms/step - loss: 45.7822 - auc: 0.7590 - accuracy: 0.8118
Epoch 6/10
533/533 [==============================] - 153s 286ms/step - loss: 39.0629 - auc: 0.7779 - accuracy: 0.8240
Epoch 7/10
533/533 [==============================] - 153s 286ms/step - loss: 32.1342 - auc: 0.8106 - accuracy: 0.8487
Epoch 8/10
533/533 [==============================] - 153s 285ms/step - loss: 25.9544 - auc: 0.8306 - accuracy: 0.8593
Epoch 9/10
533/533 [=============================

In [13]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        initial_epoch=10,
                        class_weight=class_weight)

Epoch 11/25
533/533 [==============================] - 154s 286ms/step - loss: 14.8086 - auc: 0.8846 - accuracy: 0.8963
Epoch 12/25
533/533 [==============================] - 153s 286ms/step - loss: 15.4249 - auc: 0.8774 - accuracy: 0.8910
Epoch 13/25
533/533 [==============================] - 153s 286ms/step - loss: 11.6201 - auc: 0.9081 - accuracy: 0.9116
Epoch 14/25
533/533 [==============================] - 154s 286ms/step - loss: 9.5923 - auc: 0.9162 - accuracy: 0.9149
Epoch 15/25
533/533 [==============================] - 153s 285ms/step - loss: 9.4777 - auc: 0.9153 - accuracy: 0.9153
Epoch 16/25
533/533 [==============================] - 153s 285ms/step - loss: 6.9022 - auc: 0.9325 - accuracy: 0.9289
Epoch 17/25
533/533 [==============================] - 154s 286ms/step - loss: 6.3071 - auc: 0.9396 - accuracy: 0.9332
Epoch 18/25
533/533 [==============================] - 154s 286ms/step - loss: 5.2822 - auc: 0.9488 - accuracy: 0.9396
Epoch 19/25
533/533 [========================

In [14]:
!cp ./logs/checkpoints/RGB-model-09.h5 ./drive/MyDrive/checkpoints/rgb-model-09.h5
!cp ./logs/checkpoints/RGB-model-14.h5 ./drive/MyDrive/checkpoints/rgb-model-14.h5
!cp ./logs/checkpoints/RGB-model-20.h5 ./drive/MyDrive/checkpoints/rgb-model-20.h5
!cp ./logs/checkpoints/RGB-model-25.h5 ./drive/MyDrive/checkpoints/rgb-model-25.h5

In [16]:
history = rgb_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=25,
                        class_weight=class_weight)

Epoch 26/30
533/533 [==============================] - 154s 287ms/step - loss: 1.8642 - auc: 0.9818 - accuracy: 0.9707
Epoch 27/30
533/533 [==============================] - 154s 286ms/step - loss: 1.6816 - auc: 0.9842 - accuracy: 0.9700
Epoch 28/30
533/533 [==============================] - 154s 286ms/step - loss: 1.4025 - auc: 0.9866 - accuracy: 0.9741
Epoch 29/30
533/533 [==============================] - 154s 286ms/step - loss: 1.2972 - auc: 0.9880 - accuracy: 0.9750
Epoch 30/30
533/533 [==============================] - 154s 287ms/step - loss: 1.1723 - auc: 0.9891 - accuracy: 0.9782


In [18]:
!cp ./logs/checkpoints/RGB-model-30.h5 ./drive/MyDrive/checkpoints/rgb-model-30.h5